<a href="https://colab.research.google.com/github/aswathy7576/NLP-Sarcasm-sentiment-Prediction-LDA-LSA/blob/main/LSA_%26_LDA_Classifiar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install peoplealsoask

In [ ]:

import pandas as pd
import numpy as np
import re
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, preprocess_string, strip_short, stem_text
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title
df=pd.read_json("/content/drive/MyDrive/Sarcasm_Headlines_Dataset.json", lines=True)
df.dropna(inplace=True)
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


#**RandomForestClassifier model for Classification**

In [ ]:
X = df.headline
y = df.is_sarcastic
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
print("Train set has total {0} entries with {1:.2f}% Not Sarcasm, {2:.2f}% Sarcasm".format(len(X_train),
                                                                             (len(X_train[y_train == 0]) / (len(X_train)*1.))*100,
                                                                            (len(X_train[y_train == 1]) / (len(X_train)*1.))*100))

print("Test set has total {0} entries with {1:.2f}% Not Sarcasm, {2:.2f}% Sarcasm".format(len(X_test),
                                                                             (len(X_test[y_test == 0]) / (len(X_test)*1.))*100,
                                                                            (len(X_test[y_test == 1]) / (len(X_test)*1.))*100))

# model trainer
cv = CountVectorizer()
rf = RandomForestClassifier(class_weight="balanced")
n_features = np.arange(10000,30001,10000)
# set pipeline
def accuracy_summary(pipeline, X_train, y_train, X_test, y_test):
    sentiment_fit = pipeline.fit(X_train, y_train)
    y_pred = sentiment_fit.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("accuracy score: {0:.2f}%".format(accuracy*100))
    return accuracy
def nfeature_accuracy_checker(vectorizer=cv, n_features=n_features, stop_words=None, ngram_range=(1, 1), classifier=rf):
    result = []
    print(classifier)
    print("\n")
    for n in n_features:
        vectorizer.set_params(stop_words=stop_words, max_features=n, ngram_range=ngram_range)
        checker_pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', classifier)
        ])
        print("Test result for {} features".format(n))
        nfeature_accuracy = accuracy_summary(checker_pipeline, X_train, y_train, X_test, y_test)
        result.append((n,nfeature_accuracy))
    return result

tfidf = TfidfVectorizer()
print("Result for trigram with stop words (Tfidf)\n")
feature_result_tgt = nfeature_accuracy_checker(vectorizer=tfidf,ngram_range=(1, 3))

Train set has total 20031 entries with 56.14% Not Sarcasm, 43.86% Sarcasm
Test set has total 6678 entries with 55.99% Not Sarcasm, 44.01% Sarcasm
Result for trigram with stop words (Tfidf)

RandomForestClassifier(class_weight='balanced')


Test result for 10000 features
accuracy score: 81.22%
Test result for 20000 features
accuracy score: 81.19%
Test result for 30000 features
accuracy score: 81.75%


In [ ]:
cv = CountVectorizer(max_features=30000,ngram_range=(1, 3))
pipeline = Pipeline([
        ('vectorizer', cv),
        ('classifier', rf)
    ])
sentiment_fit = pipeline.fit(X_train, y_train)
y_pred = sentiment_fit.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['Not Sarcasm','Sarcasm']))

              precision    recall  f1-score   support

 Not Sarcasm       0.86      0.80      0.83      3739
     Sarcasm       0.77      0.83      0.80      2939

    accuracy                           0.81      6678
   macro avg       0.81      0.82      0.81      6678
weighted avg       0.82      0.81      0.81      6678



#**LSA Model for topic_modeling**

In [ ]:
# preprocess given text
def preprocess(text):

    # clean text based on given filters
    CUSTOM_FILTERS = [lambda x: x.lower(),
                                remove_stopwords,
                                strip_short,
                                stem_text]
    text = preprocess_string(text, CUSTOM_FILTERS)

    return text

from gensim.models import LsiModel
# topic finding
def find_topic(data):
  df=pd.DataFrame({"text":data})
  # apply function to all reviews
  data = df["text"].apply(lambda x: preprocess(x))

  # create a dictionary with the corpus
  dictionary = corpora.Dictionary(data)

  # convert corpus into a bag of words
  bow = [dictionary.doc2bow(text) for text in data]

  # find the coherence score with a different number of topics
  for i in range(2,11):
      lsi = LsiModel(bow, num_topics=i, id2word=dictionary)
      coherence_model = CoherenceModel(model=lsi, texts=data, dictionary=dictionary, coherence='c_v')
      coherence_score = coherence_model.get_coherence()

  # perform SVD on the bag of words with the LsiModel to extract 2 topics
  lsi = LsiModel(bow, num_topics=2, id2word=dictionary)
  x=" ".join(map(str,re.findall("[A-Za-z]+",lsi.print_topics(num_words=7)[0][1])))
  return x

#**Prediction Function for LSA**

In [ ]:
def Prediction_Function(Text):
  y_pred = sentiment_fit.predict([Text])

  if y_pred==0:
    print("This is not sarcasm Using LSA the topic is: "+find_topic([Text]))
  else:
    print("This is sarcasm Using LSA and the topic is: "+find_topic([Text]))

Prediction_Function("The teaser for upcoming drama Wehshi, starring Khushhal Khan as boy-turned-brute Asif, is out")

This is not sarcasm Using LSA the topic is: asif teaser upcom khushhal boy turned brut khan wehshi


# **LDA Model For topic_modeling**

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

def extract_topic(text,n_top_words):
  q=(text,)
  value={}
  lda = LatentDirichletAllocation(n_components=10, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
  tf_vectorizer = CountVectorizer(
                                max_features=50,
                                stop_words='english')
  tf = tf_vectorizer.fit_transform(q)
  tf_feature_names = tf_vectorizer.get_feature_names()
  lda.fit(tf)

  for topic_idx, topic in enumerate(lda.components_):
        count=topic_idx
        x=" ".join([tf_feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]])
        value.update({count:x})

  return value

#**Prediction Function for LDA**

In [ ]:
def Prediction_Function(Text):
  y_pred = sentiment_fit.predict([Text])

  if y_pred==0:
    print("This is not sarcasm Using LDA and the Topic is: "+extract_topic(Text,5)[0])
  else:
    print("This is sarcasm Using LDA and the Topic is: "+extract_topic(Text,5)[0])

Prediction_Function("The teaser for upcoming drama Wehshi, starring Khushhal Khan as boy-turned-brute Asif, is out")

This is not sarcasm Using LDA and the Topic is: brute asif wehshi starring turned


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
import gradio as gr

def greet(Text):
  y_pred = sentiment_fit.predict([Text])

  if y_pred==0:
    return ("This is not sarcasm"+ "\n\nUsing LSA the topic is: "+find_topic([Text])+ " \n\nUsing LDA and the Topic is: "+extract_topic(Text,7)[0])
  else:
    return ("This is sarcasm"+ "\n\nUsing LSA and the topic is: "+find_topic([Text])+ " \n\nUsing LDA and the Topic is: "+extract_topic(Text,7)[0])


demo = gr.Interface(fn=greet, inputs="text", outputs="text",title="Topic Modeling & Classification")
demo.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://23839.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7f359c264350>,
 'http://127.0.0.1:7860/',
 'https://23839.gradio.app')

In [ ]:
#THE_END